In [ ]:
library(Signac)
library(Seurat)
library(EnsDb.Mmusculus.v79)
library(ggplot2)
library(patchwork)

In [ ]:
counts <- Read10X_h5("/SGRNJ06/randd/PROJECT/scATAC/self_pipe/mouse/A1008_2_MM_YSPB_WX_D_T7_EDTA5030/outs/A1008_2_MM_YSPB_WX_D_T7_EDTA5030_filtered_peak_count.h5")
metadata <- read.csv(
  file = "/SGRNJ06/randd/PROJECT/scATAC/self_pipe/mouse/A1008_2_MM_YSPB_WX_D_T7_EDTA5030/outs/cell_qc_metrics.tsv",
  header = TRUE,
  row.names = 1,
  sep='\t'
)
brain_assay <- CreateChromatinAssay(
  counts = counts,
  sep = c("_", "_"),
  genome = "mm10",
  fragments = '/SGRNJ06/randd/PROJECT/scATAC/self_pipe/mouse/A1008_2_MM_YSPB_WX_D_T7_EDTA5030/02.atac/Result/Mapping/A1008_2_MM_YSPB_WX_D_T7_EDTA5030//fragments_corrected_dedup_count.tsv.gz',
  min.cells = 1
)

In [ ]:
brain <- CreateSeuratObject(
  counts = brain_assay,
  assay = 'peaks',
  project = 'ATAC',
  meta.data = metadata
)

In [ ]:
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)

# change to UCSC style since the data was mapped to hg19
seqlevels(annotations) <- paste0('chr', seqlevels(annotations))
genome(annotations) <- "mm10"

# add the gene information to the object
Annotation(brain) <- annotations

In [ ]:
brain = TSSEnrichment(brain, fast = FALSE)

In [ ]:
brain$TSS.enrichment

In [ ]:
brain <- RunTFIDF(brain)
brain <- FindTopFeatures(brain, min.cutoff = 'q0')
brain <- RunSVD(object = brain)

In [ ]:
brain <- RunUMAP(
  object = brain,
  reduction = 'lsi',
  dims = 2:30
)
brain <- FindNeighbors(
  object = brain,
  reduction = 'lsi',
  dims = 2:30
)
brain <- FindClusters(
  object = brain,
  algorithm = 3,
  resolution = 1.2,
  verbose = FALSE
)

DimPlot(object = brain, label = TRUE) + NoLegend()

In [ ]:
# compute gene activities
gene.activities <- GeneActivity(brain)

# add the gene activity matrix to the Seurat object as a new assay
brain[['RNA']] <- CreateAssayObject(counts = gene.activities)
brain <- NormalizeData(
  object = brain,
  assay = 'RNA',
  normalization.method = 'LogNormalize',
  scale.factor = median(brain$nCount_RNA)
)

In [ ]:
DefaultAssay(brain) <- 'RNA'

In [ ]:
saveRDS(brain, "/SGRNJ06/randd/USER/cjj/celedev/atac/20231204scrna_atac/atac.rds")

In [ ]:
brain@assays$peaks@positionEnrichment

In [ ]:
positionEnrichment = GetAssayData(object = brain, assay = "peaks", 
    slot = "positionEnrichment")

In [ ]:
enrichment.matrix <- positionEnrichment[["TSS"]]

In [ ]:
enrichment.matrix